# Precalculated data for WDPA (AOI Summaries) - March 2023
In this notebook, we create new tables containing precalculated data for WDPA. Biodiversity data and most contextual data remain the same as before, but we include a new indicator (SPS) and new human pressure data. The project in ArcPro in which these new data was calculated is in the Virtual Machine (AOI_Summaries_Precalculated).  
The new fields included are:
- Global SPS values (SPS_global), stored in the species lookup tables 
- SPS values specific to the AOI (SPS_aoi), calculated using biodiversity data for the protected areas of each country (obtained in AOI_Summaries_Precalculations project).
- Time series human pressure data

## Table of contents
1. [Setup](#setup)
    1. [Import libraries](#libraries)
    2. [Utils](#utils)
    3. [Connect to ESRI](#esri)
2. [Prepare data](#data)
    1. [Update country names](#updatenames)
    2. [Add alternative names](#alternativenames)
    3. [Translate names to ES, FR, PT](#translatenames)
3. [Calculate biodiversity](#biodiversity)
    1. [Calculate SPS_aoi](#spsaoi)
    2. [Format biodiversity table](#biotable)
    3. [Add nspecies](#nspecies)
4. [Contextual data](#contextual)
    1. [Population and ELU](#othercontextual)
    2. [Human pressures](#pressures)


<a id='setup'></a>
## Setup

<a id='libraries'></a>
### Import libraries

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
import requests as re
from copy import deepcopy
from itertools import repeat
import functools

<a id='utils'></a>
### Utils

**getHTfromId**

In [2]:
def getHTfromId(item_id):
    item = gis.content.get(item_id)
    flayer = item.tables[0]
    sdf = flayer.query().sdf
    return sdf

**format_df**

In [3]:
def format_df(path, file_name, lookups_id):
    df = pd.read_csv(f'{path}/{file_name}')
    col_name = [col for col in df.columns if col in ['amphibians','birds','presence','reptiles']]
    df.rename(columns={'SliceNumbe':'SliceNumber',col_name[0]:'SUM'}, inplace=True)

    ### Get information from lookup tables:
    lookup = getHTfromId(lookups_id)
    df = df.merge(lookup[['SliceNumber','range_area_km2', 'SPS', 'conservation_target']], how='left',on = 'SliceNumber')
    
    ### Get species area against global species range:
    df['per_global'] = round(df['SUM']/df['range_area_km2']*100,2)
    df.loc[df['per_global']> 100,'per_global'] = 100 ### make max presence 100%
    
    ### Get species area against aoi area (this is currently not needed on the platform):
    # df = df.merge(gadm0[['MOL_ID','AREA_KM2']])
    # df['per_aoi'] = round(df['SUM']/df['AREA_KM2']*100,2)
    # df.loc[df['per_aoi']> 100,'per_aoi'] = 100 ### make max presence 100%
    
    return df

<a id='esri'></a>
### Connect to ArcGIS API

In [4]:
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

In [5]:
aol_password = env['ARCGIS_SOFIA_PASS']
aol_username = env['ARCGIS_SOFIA_USER']

In [6]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

Keyring backend being used (keyring.backends.OS_X.Keyring (priority: 5)) either failed to install or is not recommended by the keyring project (i.e. it is not secure). This means you can not use stored passwords through GIS's persistent profiles. Note that extra system-wide steps must be taken on a Linux machine to use the python keyring module securely. Read more about this at the keyring API doc (http://bit.ly/2EWDP7B) and the ArcGIS API for Python doc (http://bit.ly/2CK2wG8).


<a id='data'></a>
## Prepare data

In [7]:
path_in = '/Users/sofia/Documents/HE_Data/Precalculated/WDPA_Precalculated/Inputs'
path_out = '/Users/sofia/Documents/HE_Data/Precalculated/WDPA_Precalculated/Outputs'

In [8]:
# Import wdpa table
wdpa= gpd.read_file(f'{path_in}/WDPA_FILTERED_20210615_nomarine_wdpa_corrected_geometries/WDPA_FILTERED_20210615_nomarine_corrected_geometries.shp')
len(wdpa)

217429

In [9]:
wdpa.columns

Index(['WDPAID', 'WDPA_PID', 'PA_DEF', 'NAME', 'ORIG_NA', 'DESIG', 'DESIG_E',
       'DESIG_T', 'IUCN_CA', 'INT_CRI', 'MARINE', 'REP_M_A', 'GIS_M_A',
       'REP_ARE', 'GIS_ARE', 'NO_TAKE', 'NO_TK_A', 'STATUS', 'STATUS_',
       'GOV_TYP', 'OWN_TYP', 'MANG_AU', 'MANG_PL', 'VERIF', 'METADAT',
       'SUB_LOC', 'PARENT_', 'ISO3', 'SUPP_IN', 'CONS_OB', 'SORTER',
       'WDPA_PID_h', 'WDPA_PID__', 'hash_vl', 'AREA_KM', 'MOL_ID', 'geometry'],
      dtype='object')

In [10]:
# Create a dataframe without geometries (they will be published independently) and with only relevant columns
wdpa.rename(columns={'AREA_KM':'AREA_KM2'},inplace=True)
dff = wdpa[['MOL_ID', 'WDPAID', 'WDPA_PID', 'NAME', 'DESIG', 'DESIG_T', 'IUCN_CA',
       'STATUS', 'GOV_TYP', 'MANG_AU', 'ISO3', 'AREA_KM2', 'DESIG_E',
       'ORIG_NA', 'STATUS_']]
dff.head()

,MOL_ID,WDPAID,WDPA_PID,NAME,DESIG,DESIG_T,IUCN_CA,STATUS,GOV_TYP,MANG_AU,ISO3,AREA_KM2,DESIG_E,ORIG_NA,STATUS_
0,1,310492.0,310492,Boulder Beach,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,NZL,1.136031,Stewardship Area,Boulder Beach / WWF Block,NaN
1,2,307797.0,307797,Ferndale,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,NZL,0.748492,Scenic Reserve,Ferndale,NaN
2,3,307745.0,307745,Broughton Bay,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,NZL,0.031907,Scenic Reserve,Broughton Bay,NaN
3,4,307867.0,307867,Kaipupu Point,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,NZL,0.270855,Scenic Reserve,Kaipupu Point,NaN
4,5,303963.0,303963,Catlins Conservation Park,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,NZL,8.412168,Stewardship Area,Catlins Conservation Park,NaN


<a id='biodiversity'></a>
## Calculate biodiversity data for WDPA

In [11]:
### Ids of lookup tables for each taxa in ArcGIS online (run by Tamara in 2021)

lookups = {'amphibians':'de2309ec6aa64223a8bea682c0200d34',
         'birds':'b5f5c8d693b74abd9b0d236915d8e739',
         'mammals':'1d3b50e3b8544730ae0e2a80f00b4119',
         'reptiles':'bc6de8b9b8df4fffb6aa4208f4bf1467'}

# Get data for all taxa (calculated in arcgis pro with sample tool and saved in local)
amphibians = format_df(path_in, 'wdpa_amphibians_final_20211003.csv', lookups['amphibians'])
birds = format_df(path_in, 'wdpa_birds_final_20211003.csv', lookups['birds'])
mammals = format_df(path_in, 'wdpa_mammals_final_20211003.csv', lookups['mammals'])
reptiles = format_df(path_in, 'wdpa_reptiles_final_20211003.csv', lookups['reptiles'])

In [12]:
# Change the column name SPS to SPS_global to differenciate it from the SPS_aoi we'll calculate later
amphibians = amphibians.rename(columns = {'SPS': 'SPS_global'})
birds = birds.rename(columns = {'SPS': 'SPS_global'})
mammals = mammals.rename(columns = {'SPS': 'SPS_global'})
reptiles = reptiles.rename(columns = {'SPS': 'SPS_global'})

In [13]:
amphibians.head()

,OBJECTID,MOL_ID,X,Y,SUM,SliceNumber,Dimensions,range_area_km2,SPS_global,conservation_target,per_global
0,1,102,173.412225,-41.161038,1.0,3318,SliceNumber,296432,95,15,0.0
1,2,102,173.412225,-41.161038,1.0,3393,SliceNumber,471493,68,15,0.0
2,3,103,173.046567,-40.963196,1.0,3318,SliceNumber,296432,95,15,0.0
3,4,104,172.595189,-41.794691,1.0,3318,SliceNumber,296432,95,15,0.0
4,5,104,172.595189,-41.794691,1.0,3393,SliceNumber,471493,68,15,0.0


<a id='spsaoi'></a>
### Calculate SPS_aoi
The SPS_aoi in protected areas is, by definition, always 100% because all species found within the aoi are protected in the aoi

In [14]:
amphibians['SPS_aoi'] = 100
birds['SPS_aoi'] = 100
mammals['SPS_aoi'] = 100
reptiles['SPS_aoi'] = 100
amphibians.head()

,OBJECTID,MOL_ID,X,Y,SUM,SliceNumber,Dimensions,range_area_km2,SPS_global,conservation_target,per_global,SPS_aoi
0,1,102,173.412225,-41.161038,1.0,3318,SliceNumber,296432,95,15,0.0,100
1,2,102,173.412225,-41.161038,1.0,3393,SliceNumber,471493,68,15,0.0,100
2,3,103,173.046567,-40.963196,1.0,3318,SliceNumber,296432,95,15,0.0,100
3,4,104,172.595189,-41.794691,1.0,3318,SliceNumber,296432,95,15,0.0,100
4,5,104,172.595189,-41.794691,1.0,3393,SliceNumber,471493,68,15,0.0,100


<a id='biotable'></a>
### Format table with biodiversity data for regions

In [15]:
# Format biodiversity data in a string
amphibians_bio = amphibians.groupby('MOL_ID')[['SliceNumber', 'per_global', 'SPS_global', 'SPS_aoi']].apply(lambda x: x.to_json(orient='records')).to_frame('amphibians').reset_index()
birds_bio = birds.groupby('MOL_ID')[['SliceNumber', 'per_global', 'SPS_global', 'SPS_aoi']].apply(lambda x: x.to_json(orient='records')).to_frame('birds').reset_index()
mammals_bio = mammals.groupby('MOL_ID')[['SliceNumber', 'per_global', 'SPS_global', 'SPS_aoi']].apply(lambda x: x.to_json(orient='records')).to_frame('mammals').reset_index()
reptiles_bio = reptiles.groupby('MOL_ID')[['SliceNumber', 'per_global', 'SPS_global', 'SPS_aoi']].apply(lambda x: x.to_json(orient='records')).to_frame('reptiles').reset_index()


In [16]:
amphibians_bio

,MOL_ID,amphibians
0,2,"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo..."
1,3,"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo..."
2,4,"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo..."
3,5,"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo..."
4,6,"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo..."
...,...,...
193873,217452,"[{""SliceNumber"":3222,""per_global"":0.0,""SPS_glo..."
193874,217453,"[{""SliceNumber"":3222,""per_global"":0.0,""SPS_glo..."
193875,217455,"[{""SliceNumber"":3222,""per_global"":0.0,""SPS_glo..."
193876,217457,"[{""SliceNumber"":168,""per_global"":0.0,""SPS_glob..."


In [17]:
dff = pd.merge(dff, amphibians_bio, how='left', on = 'MOL_ID')
dff = pd.merge(dff, birds_bio, how='left', on = 'MOL_ID')
dff = pd.merge(dff, mammals_bio, how='left', on = 'MOL_ID')
dff = pd.merge(dff, reptiles_bio, how='left', on = 'MOL_ID')
dff.head()

,MOL_ID,WDPAID,WDPA_PID,NAME,DESIG,DESIG_T,IUCN_CA,STATUS,GOV_TYP,MANG_AU,ISO3,AREA_KM2,DESIG_E,ORIG_NA,STATUS_,amphibians,birds,mammals,reptiles
0,1,310492.0,310492,Boulder Beach,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,NZL,1.136031,Stewardship Area,Boulder Beach / WWF Block,NaN,NaN,"[{""SliceNumber"":482.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...",NaN
1,2,307797.0,307797,Ferndale,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,NZL,0.748492,Scenic Reserve,Ferndale,NaN,"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo...","[{""SliceNumber"":8.0,""per_global"":0.0,""SPS_glob...","[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":6163,""per_global"":0.0,""SPS_glo..."
2,3,307745.0,307745,Broughton Bay,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,NZL,0.031907,Scenic Reserve,Broughton Bay,NaN,"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo...","[{""SliceNumber"":1847.0,""per_global"":0.0,""SPS_g...","[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":6499,""per_global"":0.0,""SPS_glo..."
3,4,307867.0,307867,Kaipupu Point,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,NZL,0.270855,Scenic Reserve,Kaipupu Point,NaN,"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo...","[{""SliceNumber"":8.0,""per_global"":0.0,""SPS_glob...","[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":6163,""per_global"":0.0,""SPS_glo..."
4,5,303963.0,303963,Catlins Conservation Park,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,NZL,8.412168,Stewardship Area,Catlins Conservation Park,NaN,"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo...","[{""SliceNumber"":8.0,""per_global"":0.01,""SPS_glo...",NaN,"[{""SliceNumber"":6163,""per_global"":0.01,""SPS_gl..."


In [18]:
dff = dff.fillna('[]')
dff.head()

,MOL_ID,WDPAID,WDPA_PID,NAME,DESIG,DESIG_T,IUCN_CA,STATUS,GOV_TYP,MANG_AU,ISO3,AREA_KM2,DESIG_E,ORIG_NA,STATUS_,amphibians,birds,mammals,reptiles
0,1,310492.0,310492,Boulder Beach,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,NZL,1.136031,Stewardship Area,Boulder Beach / WWF Block,[],[],"[{""SliceNumber"":482.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...",[]
1,2,307797.0,307797,Ferndale,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,NZL,0.748492,Scenic Reserve,Ferndale,[],"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo...","[{""SliceNumber"":8.0,""per_global"":0.0,""SPS_glob...","[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":6163,""per_global"":0.0,""SPS_glo..."
2,3,307745.0,307745,Broughton Bay,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,NZL,0.031907,Scenic Reserve,Broughton Bay,[],"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo...","[{""SliceNumber"":1847.0,""per_global"":0.0,""SPS_g...","[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":6499,""per_global"":0.0,""SPS_glo..."
3,4,307867.0,307867,Kaipupu Point,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,NZL,0.270855,Scenic Reserve,Kaipupu Point,[],"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo...","[{""SliceNumber"":8.0,""per_global"":0.0,""SPS_glob...","[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":6163,""per_global"":0.0,""SPS_glo..."
4,5,303963.0,303963,Catlins Conservation Park,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,NZL,8.412168,Stewardship Area,Catlins Conservation Park,[],"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo...","[{""SliceNumber"":8.0,""per_global"":0.01,""SPS_glo...",[],"[{""SliceNumber"":6163,""per_global"":0.01,""SPS_gl..."


In [19]:
dff.loc[dff['MOL_ID']==121,'birds'].values[0]

'[{"SliceNumber":251.0,"per_global":0.0,"SPS_global":57,"SPS_aoi":100},{"SliceNumber":552.0,"per_global":0.0,"SPS_global":100,"SPS_aoi":100},{"SliceNumber":613.0,"per_global":0.0,"SPS_global":100,"SPS_aoi":100},{"SliceNumber":1301.0,"per_global":0.0,"SPS_global":40,"SPS_aoi":100},{"SliceNumber":1310.0,"per_global":0.0,"SPS_global":67,"SPS_aoi":100},{"SliceNumber":1321.0,"per_global":0.0,"SPS_global":100,"SPS_aoi":100},{"SliceNumber":1510.0,"per_global":0.0,"SPS_global":82,"SPS_aoi":100},{"SliceNumber":1511.0,"per_global":0.0,"SPS_global":95,"SPS_aoi":100},{"SliceNumber":1517.0,"per_global":0.0,"SPS_global":74,"SPS_aoi":100},{"SliceNumber":1572.0,"per_global":0.0,"SPS_global":58,"SPS_aoi":100},{"SliceNumber":1933.0,"per_global":0.0,"SPS_global":75,"SPS_aoi":100},{"SliceNumber":2017.0,"per_global":0.0,"SPS_global":85,"SPS_aoi":100},{"SliceNumber":3025.0,"per_global":0.0,"SPS_global":53,"SPS_aoi":100},{"SliceNumber":3027.0,"per_global":0.0,"SPS_global":88,"SPS_aoi":100},{"SliceNumber":308

In [20]:
len(dff)

217429

<a id='nspecies'></a>
### Add nspecies

In [21]:
# Get data for all taxa
a = pd.read_csv(f'{path_in}/wdpa_amphibians_final_20211003.csv')
b = pd.read_csv(f'{path_in}/wdpa_birds_final_20211003.csv')
m = pd.read_csv(f'{path_in}/wdpa_mammals_final_20211003.csv')
r = pd.read_csv(f'{path_in}/wdpa_reptiles_final_20211003.csv')

In [22]:
# Count number of species for group
a_count = a.groupby('MOL_ID')['SliceNumber'].count().astype(int)
b_count = b.groupby('MOL_ID')['SliceNumber'].count().astype(int)
m_count = m.groupby('MOL_ID')['SliceNumber'].count().astype(int)
r_count = r.groupby('MOL_ID')['SliceNumber'].count().astype(int)

In [23]:
frame = { 'amph_nspecies': a_count, 'bird_nspecies': b_count, 'mamm_nspecies': m_count, 'rept_nspecies': r_count }
df = pd.DataFrame(frame).reset_index()
cols = ['amph_nspecies', 'bird_nspecies', 'mamm_nspecies', 'rept_nspecies']
df[cols] = df[cols].fillna(0)
df[cols] = df[cols].astype('int')
df['nspecies'] = df['amph_nspecies'] + df['bird_nspecies'] + df['mamm_nspecies'] + df['rept_nspecies']
df

,MOL_ID,amph_nspecies,bird_nspecies,mamm_nspecies,rept_nspecies,nspecies
0,1,0,49,3,0,52
1,2,2,68,2,6,78
2,3,2,24,2,2,30
3,4,2,62,2,4,70
4,5,2,48,0,3,53
...,...,...,...,...,...,...
205893,217453,2,95,25,2,124
205894,217455,2,76,25,2,105
205895,217456,0,40,2,0,42
205896,217457,6,59,40,3,108


In [24]:
# Merge nspecies in dataframe
wdpa_nspecies = dff.merge(df, how='left', on = 'MOL_ID')
wdpa_nspecies

,MOL_ID,WDPAID,WDPA_PID,NAME,DESIG,DESIG_T,IUCN_CA,STATUS,GOV_TYP,MANG_AU,...,STATUS_,amphibians,birds,mammals,reptiles,amph_nspecies,bird_nspecies,mamm_nspecies,rept_nspecies,nspecies
0,1,310492.0,310492,Boulder Beach,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,[],[],"[{""SliceNumber"":482.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...",[],0.0,49.0,3.0,0.0,52.0
1,2,307797.0,307797,Ferndale,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,[],"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo...","[{""SliceNumber"":8.0,""per_global"":0.0,""SPS_glob...","[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":6163,""per_global"":0.0,""SPS_glo...",2.0,68.0,2.0,6.0,78.0
2,3,307745.0,307745,Broughton Bay,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,[],"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo...","[{""SliceNumber"":1847.0,""per_global"":0.0,""SPS_g...","[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":6499,""per_global"":0.0,""SPS_glo...",2.0,24.0,2.0,2.0,30.0
3,4,307867.0,307867,Kaipupu Point,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,[],"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo...","[{""SliceNumber"":8.0,""per_global"":0.0,""SPS_glob...","[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":6163,""per_global"":0.0,""SPS_glo...",2.0,62.0,2.0,4.0,70.0
4,5,303963.0,303963,Catlins Conservation Park,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,[],"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo...","[{""SliceNumber"":8.0,""per_global"":0.01,""SPS_glo...",[],"[{""SliceNumber"":6163,""per_global"":0.01,""SPS_gl...",2.0,48.0,0.0,3.0,53.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217424,217454,555682973.0,555682973,4.11 St Sampson's Marais / Ivy Castle,Sites of Special Significance,National,OECM,Designated,Federal or national ministry or agency,States of Guernsey and private land owners,...,2016.0,[],[],[],[],NaN,NaN,NaN,NaN,NaN
217425,217455,555682979.0,555682979,4.7 Les Vicheries and Rue Rocheuse,Sites of Special Significance,National,OECM,Designated,Federal or national ministry or agency,Private land owners,...,2016.0,"[{""SliceNumber"":3222,""per_global"":0.0,""SPS_glo...","[{""SliceNumber"":120.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":624.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":4843,""per_global"":0.0,""SPS_glo...",2.0,76.0,25.0,2.0,105.0
217426,217456,555651689.0,555651689,"Les Demoiselles nursery (Plaisance Bay), Magda...",Other Effective Area-Based Conservation Measure,National,OECM,Designated,Federal or national ministry or agency,Fisheries And Oceans Canada,...,2016.0,[],"[{""SliceNumber"":142.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":2954.0,""per_global"":0.0,""SPS_g...",[],0.0,40.0,2.0,0.0,42.0
217427,217457,555651698.0,555651698,Strait Of Georgia And Howe Sound Glass Sponge ...,Other Effective Area-Based Conservation Measure,National,OECM,Designated,Federal or national ministry or agency,Fisheries And Oceans Canada,...,2019.0,"[{""SliceNumber"":168,""per_global"":0.0,""SPS_glob...","[{""SliceNumber"":142.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":556.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":2215,""per_global"":0.0,""SPS_glo...",6.0,59.0,40.0,3.0,108.0


In [25]:
wdpa_nspecies[wdpa_nspecies.nspecies.isna()]

,MOL_ID,WDPAID,WDPA_PID,NAME,DESIG,DESIG_T,IUCN_CA,STATUS,GOV_TYP,MANG_AU,...,STATUS_,amphibians,birds,mammals,reptiles,amph_nspecies,bird_nspecies,mamm_nspecies,rept_nspecies,nspecies
24,25,307839.0,307839,Henley,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,[],[],[],[],[],NaN,NaN,NaN,NaN,NaN
40,41,555566139.0,555566139,Tavora Reserve,Conservation Covenant,National,IV,Designated,Collaborative governance,Collaborative,...,[],[],[],[],[],NaN,NaN,NaN,NaN,NaN
47,48,555566148.0,555566148,Nardoo Stream (Waipori),Conservation Covenant,National,IV,Designated,Collaborative governance,Collaborative,...,[],[],[],[],[],NaN,NaN,NaN,NaN,NaN
56,57,555566173.0,555566173,Karoro Creek (Kitto),Conservation Covenant,National,IV,Designated,Collaborative governance,Collaborative,...,[],[],[],[],[],NaN,NaN,NaN,NaN,NaN
70,71,555566192.0,555566192,JO & AP Lewis - Kotinga,Conservation Covenant,National,IV,Designated,Collaborative governance,Collaborative,...,[],[],[],[],[],NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217349,217379,555651798.0,555651798,Earl's Place,Heritage Conservation Zone in regional land us...,National,OECM,Designated,Collaborative governance,"Indigenous Government, Government Of The North...",...,2003.0,[],[],[],[],NaN,NaN,NaN,NaN,NaN
217362,217392,555651723.0,555651723,Lobster Area Closure (Mouse Island),Other Effective Area-Based Conservation Measure,National,OECM,Designated,Federal or national ministry or agency,Fisheries And Oceans Canada,...,2004.0,[],[],[],[],NaN,NaN,NaN,NaN,NaN
217417,217447,555682976.0,555682976,4.3 Fort Hommet & Vazon coast,Sites of Special Significance,National,OECM,Designated,Federal or national ministry or agency,States of Guernsey,...,2016.0,[],[],[],[],NaN,NaN,NaN,NaN,NaN
217420,217450,555682980.0,555682980,4.8 Lihou Island,Sites of Special Significance,National,OECM,Designated,Federal or national ministry or agency,States of Guernsey,...,2016.0,[],[],[],[],NaN,NaN,NaN,NaN,NaN


**Save table with biodiversity data**

In [26]:
wdpa_nspecies.to_csv((f'{path_out}/wdpa_precalculated_SPS_biodiversity_only.csv'))

<a id='contextual'></a>
## Add contextual data
Population and ELU come from previous calculations. In this iteration we include the new human modification time series.

<a id='othercontextual'></a>
### Population and ELU

In [27]:
pop = pd.read_csv(f'{path_in}/Pop.csv')
elu = pd.read_csv(f'{path_in}/ELU.csv')

In [29]:
## Add contextual data: ELU
dff2 = wdpa_nspecies.merge(elu[['MOL_ID','MAJORITY']], how='left', on = 'MOL_ID').rename(columns={'MAJORITY':'majority_land_cover_climate_regime'})
dff2.head()

,MOL_ID,WDPAID,WDPA_PID,NAME,DESIG,DESIG_T,IUCN_CA,STATUS,GOV_TYP,MANG_AU,...,amphibians,birds,mammals,reptiles,amph_nspecies,bird_nspecies,mamm_nspecies,rept_nspecies,nspecies,majority_land_cover_climate_regime
0,1,310492.0,310492,Boulder Beach,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,[],"[{""SliceNumber"":482.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...",[],0.0,49.0,3.0,0.0,52.0,107.0
1,2,307797.0,307797,Ferndale,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo...","[{""SliceNumber"":8.0,""per_global"":0.0,""SPS_glob...","[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":6163,""per_global"":0.0,""SPS_glo...",2.0,68.0,2.0,6.0,78.0,176.0
2,3,307745.0,307745,Broughton Bay,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo...","[{""SliceNumber"":1847.0,""per_global"":0.0,""SPS_g...","[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":6499,""per_global"":0.0,""SPS_glo...",2.0,24.0,2.0,2.0,30.0,NaN
3,4,307867.0,307867,Kaipupu Point,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo...","[{""SliceNumber"":8.0,""per_global"":0.0,""SPS_glob...","[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":6163,""per_global"":0.0,""SPS_glo...",2.0,62.0,2.0,4.0,70.0,NaN
4,5,303963.0,303963,Catlins Conservation Park,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,"[{""SliceNumber"":3318,""per_global"":0.0,""SPS_glo...","[{""SliceNumber"":8.0,""per_global"":0.01,""SPS_glo...",[],"[{""SliceNumber"":6163,""per_global"":0.01,""SPS_gl...",2.0,48.0,0.0,3.0,53.0,97.0


In [32]:
# Retrieve elu lookup table to see to see the correspondences for that elu code
elu_lookup = getHTfromId('83802a7fa3d34c1fa40844fc14683966')
elu_lookup.head()

,elu_code,elu,lc_type,lf_type,cr_type,ObjectId
0,301,Sub Tropical Moist Forest on Plains,Forest,Plains,Sub Tropical Moist,1
1,201,Warm Temperate Dry Sparsley or Non vegetated o...,Sparsley or Non vegetated,Plains,Warm Temperate Dry,2
2,151,Cool Temperate Dry Sparsley or Non vegetated o...,Sparsley or Non vegetated,Plains,Cool Temperate Dry,3
3,302,Sub Tropical Moist Cropland on Tablelands,Cropland,Tablelands,Sub Tropical Moist,4
4,152,Cool Temperate Dry Sparsley or Non vegetated o...,Sparsley or Non vegetated,Tablelands,Cool Temperate Dry,5


In [33]:
# Merge in dataset the required info from lookup table
dff2 = dff2.merge(elu_lookup[['elu_code','lc_type','cr_type']], how='left', left_on = 'majority_land_cover_climate_regime', right_on = 'elu_code')\
    .drop(columns=['elu_code'])\
    .rename(columns={'lc_type':'land_cover_majority','cr_type':'climate_regime_majority'})
dff2.head()

,MOL_ID,WDPAID,WDPA_PID,NAME,DESIG,DESIG_T,IUCN_CA,STATUS,GOV_TYP,MANG_AU,...,mammals,reptiles,amph_nspecies,bird_nspecies,mamm_nspecies,rept_nspecies,nspecies,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority
0,1,310492.0,310492,Boulder Beach,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,"[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...",[],0.0,49.0,3.0,0.0,52.0,107.0,Grassland,Cool Temperate Moist
1,2,307797.0,307797,Ferndale,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,"[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":6163,""per_global"":0.0,""SPS_glo...",2.0,68.0,2.0,6.0,78.0,176.0,Forest,Warm Temperate Moist
2,3,307745.0,307745,Broughton Bay,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,"[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":6499,""per_global"":0.0,""SPS_glo...",2.0,24.0,2.0,2.0,30.0,NaN,NaN,NaN
3,4,307867.0,307867,Kaipupu Point,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,"[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":6163,""per_global"":0.0,""SPS_glo...",2.0,62.0,2.0,4.0,70.0,NaN,NaN,NaN
4,5,303963.0,303963,Catlins Conservation Park,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,[],"[{""SliceNumber"":6163,""per_global"":0.01,""SPS_gl...",2.0,48.0,0.0,3.0,53.0,97.0,Forest,Cool Temperate Moist


In [34]:
dff2['land_cover_majority'] = dff2['land_cover_majority'].fillna('')
dff2['climate_regime_majority'] = dff2['climate_regime_majority'].fillna('')
dff2 = dff2.fillna(0)
dff2.head()

,MOL_ID,WDPAID,WDPA_PID,NAME,DESIG,DESIG_T,IUCN_CA,STATUS,GOV_TYP,MANG_AU,...,mammals,reptiles,amph_nspecies,bird_nspecies,mamm_nspecies,rept_nspecies,nspecies,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority
0,1,310492.0,310492,Boulder Beach,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,"[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...",[],0.0,49.0,3.0,0.0,52.0,107.0,Grassland,Cool Temperate Moist
1,2,307797.0,307797,Ferndale,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,"[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":6163,""per_global"":0.0,""SPS_glo...",2.0,68.0,2.0,6.0,78.0,176.0,Forest,Warm Temperate Moist
2,3,307745.0,307745,Broughton Bay,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,"[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":6499,""per_global"":0.0,""SPS_glo...",2.0,24.0,2.0,2.0,30.0,0.0,,
3,4,307867.0,307867,Kaipupu Point,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,"[{""SliceNumber"":303.0,""per_global"":0.0,""SPS_gl...","[{""SliceNumber"":6163,""per_global"":0.0,""SPS_glo...",2.0,62.0,2.0,4.0,70.0,0.0,,
4,5,303963.0,303963,Catlins Conservation Park,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,[],"[{""SliceNumber"":6163,""per_global"":0.01,""SPS_gl...",2.0,48.0,0.0,3.0,53.0,97.0,Forest,Cool Temperate Moist


In [35]:
## Add contextual data: POP
dff2 = dff2.merge(pop[['MOL_ID','SUM']],on ='MOL_ID',how='left')
dff2.head(1)

,MOL_ID,WDPAID,WDPA_PID,NAME,DESIG,DESIG_T,IUCN_CA,STATUS,GOV_TYP,MANG_AU,...,reptiles,amph_nspecies,bird_nspecies,mamm_nspecies,rept_nspecies,nspecies,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,SUM
0,1,310492.0,310492,Boulder Beach,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,[],0.0,49.0,3.0,0.0,52.0,107.0,Grassland,Cool Temperate Moist,2.110001


In [36]:
dff2 = dff2.rename(columns ={'SUM':'population_sum'})
dff2.head(1)

,MOL_ID,WDPAID,WDPA_PID,NAME,DESIG,DESIG_T,IUCN_CA,STATUS,GOV_TYP,MANG_AU,...,reptiles,amph_nspecies,bird_nspecies,mamm_nspecies,rept_nspecies,nspecies,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,population_sum
0,1,310492.0,310492,Boulder Beach,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,[],0.0,49.0,3.0,0.0,52.0,107.0,Grassland,Cool Temperate Moist,2.110001


<a id='pressures'></a>
### Human pressures 

In [74]:
# Bring new human pressure tables
agriculture = pd.read_csv(f'{path_in}/Contextual_wdpa_new/HP_wdpa_agriculture_table.csv')
builtup = pd.read_csv(f'{path_in}/Contextual_wdpa_new/HP_wdpa_builtup_table.csv')
extraction = pd.read_csv(f'{path_in}/Contextual_wdpa_new/HP_wdpa_extraction_table.csv')
intrusion = pd.read_csv(f'{path_in}/Contextual_wdpa_new/HP_wdpa_intrusion_table.csv')
transportation = pd.read_csv(f'{path_in}/Contextual_wdpa_new/HP_wdpa_transportation_table.csv')


#### Note

Some of the WDPA have "Null" values for the different human pressure classes although they should have values. This is due to a problem with the  Zonal Statistics tool in ArcPro that produced the folloing warning: `WARNING 002858: Certain rows set to NULL due to error while evaluating python expression: TypeError: unsupported operand type(s) for /: 'float' and 'NoneType'.` 

In [43]:
# # Select all MOL_ID that are Null in each of the human pressure tables
# agr_null = list(agriculture.MOL_ID[agriculture.percentage_land_encroachment.isnull()].unique())
# bui_null = list(builtup.MOL_ID[builtup.percentage_land_encroachment.isnull()].unique())
# ext_null = list(extraction.MOL_ID[extraction.percentage_land_encroachment.isnull()].unique())
# int_null = list(intrusion.MOL_ID[intrusion.percentage_land_encroachment.isnull()].unique())
# tra_null = list(transportation.MOL_ID[transportation.percentage_land_encroachment.isnull()].unique())

In [44]:
# Select and export wdpa that have Null values in each of the human pressure tables, to rerun the tool in ArcPro for those
# wdpa_null_agr = wdpa[wdpa['MOL_ID'].isin(agr_null)]
# wdpa_null_bui = wdpa[wdpa['MOL_ID'].isin(bui_null)]
# wdpa_null_ext = wdpa[wdpa['MOL_ID'].isin(ext_null)]
# wdpa_null_int = wdpa[wdpa['MOL_ID'].isin(int_null)]
# wdpa_null_tra = wdpa[wdpa['MOL_ID'].isin(tra_null)]

# wdpa_null_agr.to_file(f'{path_out}/Nulls/wdpa_null_agr.shp', driver= 'ESRI Shapefile')
# wdpa_null_bui.to_file(f'{path_out}/Nulls/wdpa_null_bui.shp', driver= 'ESRI Shapefile')
# wdpa_null_ext.to_file(f'{path_out}/Nulls/wdpa_null_ext.shp', driver= 'ESRI Shapefile')
# wdpa_null_int.to_file(f'{path_out}/Nulls/wdpa_null_int.shp', driver= 'ESRI Shapefile')
# wdpa_null_tra.to_file(f'{path_out}/Nulls/wdpa_null_tra.shp', driver= 'ESRI Shapefile')

In [77]:
# Import new human pressure tables
# agriculture2 = pd.read_csv(f'{path_in}/Contextual_wdpa_new/HP_wdpa2_agriculture_table.csv')
# agriculture3 = pd.read_csv(f'{path_in}/Contextual_wdpa_new/HP_wdpa3_agriculture_table.csv')
# agriculture4 = pd.read_csv(f'{path_in}/Contextual_wdpa_new/HP_wdpa4_agriculture_table.csv')

In [80]:
# Take only relevant columns from datasets
# agriculture = agriculture[['MOL_ID', 'Year', 'percentage_land_encroachment']].astype({'Year':'int'})
# agriculture2 = agriculture2[['MOL_ID', 'Year', 'percentage_land_encroachment']].astype({'Year':'int'})
# agriculture3 = agriculture3[['MOL_ID', 'Year', 'percentage_land_encroachment']].astype({'Year':'int'})
# agriculture4 = agriculture4[['MOL_ID', 'Year', 'percentage_land_encroachment']].astype({'Year':'int'})

In [103]:
# Concatenate all dataframes for human pressure in one
# frames = [agriculture, agriculture2, agriculture3, agriculture4]
# agriculture_final = pd.concat(frames)
# agriculture_final.head(10)

,MOL_ID,Year,percentage_land_encroachment
0,263,1990,0.23
1,277,1990,100.00
2,286,1990,101.47
3,287,1990,96.08
4,288,1990,103.61
5,294,1990,88.78
6,299,1990,101.43
7,300,1990,97.87
8,303,1990,90.00
9,304,1990,116.67


In [105]:
# Limit results over 100 to 100%
# agriculture_final.loc[agriculture_final['percentage_land_encroachment']> 100,'percentage_land_encroachment'] = 100 ### make max presence 100%
# agriculture_final.head(10)

,MOL_ID,Year,percentage_land_encroachment
0,263,1990,0.23
1,277,1990,100.00
2,286,1990,100.00
3,287,1990,96.08
4,288,1990,100.00
5,294,1990,88.78
6,299,1990,100.00
7,300,1990,97.87
8,303,1990,90.00
9,304,1990,100.00


In [106]:
# Format it
# agr = agriculture_final.groupby('MOL_ID')[['Year', 'percentage_land_encroachment']].apply(lambda x: x.to_json(orient='records')).to_frame('agriculture').reset_index()
# agr.head(1)

,MOL_ID,agriculture
0,220,"[{""Year"":2005,""percentage_land_encroachment"":2..."


#### Format human pressures

In [ ]:
agriculture = agriculture[['MOL_ID', 'Year', 'percentage_land_encroachment']].astype({'Year':'int'})
builtup = builtup[['MOL_ID', 'Year', 'percentage_land_encroachment']].astype({'Year':'int'})
extraction = extraction[['MOL_ID', 'Year', 'percentage_land_encroachment']].astype({'Year':'int'})
intrusion = intrusion[['MOL_ID', 'Year', 'percentage_land_encroachment']].astype({'Year':'int'})
transportation = transportation[['MOL_ID', 'Year', 'percentage_land_encroachment']].astype({'Year':'int'})

In [55]:
# Format them to have required fields in a string
agr = agriculture.groupby('MOL_ID')[['Year', 'percentage_land_encroachment']].apply(lambda x: x.to_json(orient='records')).to_frame('agriculture').reset_index()
bui = builtup.groupby('MOL_ID')[['Year', 'percentage_land_encroachment']].apply(lambda x: x.to_json(orient='records')).to_frame('builtup').reset_index()
ext = extraction.groupby('MOL_ID')[['Year', 'percentage_land_encroachment']].apply(lambda x: x.to_json(orient='records')).to_frame('extraction').reset_index()
int = intrusion.groupby('MOL_ID')[['Year', 'percentage_land_encroachment']].apply(lambda x: x.to_json(orient='records')).to_frame('intrusion').reset_index()
tra = transportation.groupby('MOL_ID')[['Year', 'percentage_land_encroachment']].apply(lambda x: x.to_json(orient='records')).to_frame('transportation').reset_index()

In [59]:
agr.agriculture[0]

'[{"Year":2005,"percentage_land_encroachment":21.13},{"Year":2010,"percentage_land_encroachment":81.22},{"Year":2015,"percentage_land_encroachment":95.86},{"Year":2017,"percentage_land_encroachment":97.68}]'

In [108]:
ctx_all = pd.merge(dff2, agr, how='left', on = 'MOL_ID')
ctx_all = pd.merge(ctx_all, bui, how='left', on = 'MOL_ID')
ctx_all = pd.merge(ctx_all, ext, how='left', on = 'MOL_ID')
ctx_all = pd.merge(ctx_all, int, how='left', on = 'MOL_ID')
ctx_all = pd.merge(ctx_all, tra, how='left', on = 'MOL_ID')
ctx_all.head(10)

,MOL_ID,WDPAID,WDPA_PID,NAME,DESIG,DESIG_T,IUCN_CA,STATUS,GOV_TYP,MANG_AU,...,nspecies,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,population_sum,agriculture,builtup,extraction,intrusion,transportation
0,1,310492.0,310492,Boulder Beach,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,52.0,107.0,Grassland,Cool Temperate Moist,2.110001,NaN,NaN,NaN,NaN,NaN
1,2,307797.0,307797,Ferndale,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,78.0,176.0,Forest,Warm Temperate Moist,1.315837,NaN,NaN,NaN,NaN,NaN
2,3,307745.0,307745,Broughton Bay,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,30.0,0.0,,,NaN,NaN,NaN,NaN,NaN,NaN
3,4,307867.0,307867,Kaipupu Point,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,70.0,0.0,,,NaN,NaN,NaN,NaN,NaN,NaN
4,5,303963.0,303963,Catlins Conservation Park,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,53.0,97.0,Forest,Cool Temperate Moist,3.103363,NaN,NaN,NaN,NaN,NaN
5,6,555565660.0,555565660,Mt Aspiring/Tititea,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,53.0,97.0,Forest,Cool Temperate Moist,0.224189,NaN,NaN,NaN,NaN,NaN
6,7,555564797.0,555564797,Kenepuru Sound,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,52.0,176.0,Forest,Warm Temperate Moist,1.076915,NaN,NaN,NaN,NaN,NaN
7,8,310790.0,310790,Four Rivers Plain,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,42.0,171.0,Grassland,Warm Temperate Moist,0.472719,NaN,NaN,NaN,NaN,NaN
8,9,555566679.0,555566679,Earnscleugh - Rough Creek,Conservation Covenant,National,IV,Designated,Collaborative governance,Collaborative,...,23.0,107.0,Grassland,Cool Temperate Moist,0.222667,NaN,NaN,NaN,NaN,NaN
9,10,555564992.0,555564992,Fyfe Cottage Exchange,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,49.0,0.0,,,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
nulls = ctx_all[ctx_all[['agriculture', 'builtup','extraction', 'intrusion', 'transportation']].isna().all(axis=1)]
nulls

,MOL_ID,WDPAID,WDPA_PID,NAME,DESIG,DESIG_T,IUCN_CA,STATUS,GOV_TYP,MANG_AU,...,nspecies,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,population_sum,agriculture,builtup,extraction,intrusion,transportation
0,1,310492.0,310492,Boulder Beach,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,52.0,107.0,Grassland,Cool Temperate Moist,2.110001,NaN,NaN,NaN,NaN,NaN
1,2,307797.0,307797,Ferndale,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,78.0,176.0,Forest,Warm Temperate Moist,1.315837,NaN,NaN,NaN,NaN,NaN
2,3,307745.0,307745,Broughton Bay,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,30.0,0.0,,,NaN,NaN,NaN,NaN,NaN,NaN
3,4,307867.0,307867,Kaipupu Point,Scenic Reserve,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,70.0,0.0,,,NaN,NaN,NaN,NaN,NaN,NaN
4,5,303963.0,303963,Catlins Conservation Park,Stewardship Area,National,III,Designated,Federal or national ministry or agency,Department of Conservation,...,53.0,97.0,Forest,Cool Temperate Moist,3.103363,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217423,217453,555682972.0,555682972,4.10 South Vazon / La Grande Mare wet meadows,Sites of Special Significance,National,OECM,Designated,Federal or national ministry or agency,States of Guernsey and private land owners,...,124.0,168.0,Cropland,Warm Temperate Moist,292.069336,NaN,NaN,NaN,NaN,NaN
217425,217455,555682979.0,555682979,4.7 Les Vicheries and Rue Rocheuse,Sites of Special Significance,National,OECM,Designated,Federal or national ministry or agency,Private land owners,...,105.0,168.0,Cropland,Warm Temperate Moist,145.534790,NaN,NaN,NaN,NaN,NaN
217426,217456,555651689.0,555651689,"Les Demoiselles nursery (Plaisance Bay), Magda...",Other Effective Area-Based Conservation Measure,National,OECM,Designated,Federal or national ministry or agency,Fisheries And Oceans Canada,...,42.0,0.0,,,NaN,NaN,NaN,NaN,NaN,NaN
217427,217457,555651698.0,555651698,Strait Of Georgia And Howe Sound Glass Sponge ...,Other Effective Area-Based Conservation Measure,National,OECM,Designated,Federal or national ministry or agency,Fisheries And Oceans Canada,...,108.0,0.0,,,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
molID_null = list(nulls.MOL_ID)
len(molID_null)

140813

In [115]:
wdpa= gpd.read_file(f'{path_in}/WDPA_FILTERED_20210615_FILTERED_nomarine_with_oecm_wdpa/WDPA_FILTERED_20210615_FILTERED_nomarine_with_oecm_wdpa.shp')
null_hp = wdpa[wdpa['MOL_ID'].isin(molID_null)]
len(null_hp)

140813

In [116]:
null_hp = null_hp[['MOL_ID', 'geometry']]
null_hp.to_file(f'{path_out}/wdpa_null_hp.shp', driver='ESRI Shapefile')

In [ ]:
# Merge in master dataframe
dff_ctx = pd.merge(gadm1_nspecies, ctx_all, how='left', on = 'MOL_ID')
dff_ctx.head()

,GID_0,NAME_0,GID_1,NAME_1,MOL_ID,AREA_KM2,geometry,amphibians,birds,mammals,...,population_sum,majority_land_cover_climate_reg,land_cover_majority,climate_regime_majority,country_size,agriculture,builtup,extraction,intrusion,transportation
0,AFG,Afghanistan,AFG.1_1,Badakhshan,1,43692.210235,"POLYGON ((71.10155 35.95555, 71.08842 35.92924...","[{""SliceNumber"":951,""per_global"":0.15,""SPS_glo...","[{""SliceNumber"":92,""per_global"":0.1,""SPS_globa...","[{""SliceNumber"":167,""per_global"":4.13,""SPS_glo...",...,1236789.0,107.0,Grassland,Cool Temperate Moist,3,NaN,"[{""Year"":1990,""percentage_land_encroachment"":0...",NaN,NaN,NaN
1,AFG,Afghanistan,AFG.2_1,Badghis,2,20589.857163,"POLYGON ((63.09734 34.64551, 63.06237 34.69018...","[{""SliceNumber"":1707,""per_global"":0.15,""SPS_gl...","[{""SliceNumber"":26,""per_global"":0.08,""SPS_glob...","[{""SliceNumber"":575,""per_global"":0.34,""SPS_glo...",...,652350.2,205.0,Sparsley or Non vegetated,Warm Temperate Dry,3,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,AFG.3_1,Baghlan,3,21120.261382,"POLYGON ((67.35538 34.88549, 67.33750 34.91966...",NaN,"[{""SliceNumber"":26,""per_global"":0.05,""SPS_glob...","[{""SliceNumber"":167,""per_global"":2.03,""SPS_glo...",...,1040738.0,107.0,Grassland,Cool Temperate Moist,3,NaN,"[{""Year"":1990,""percentage_land_encroachment"":0...",NaN,NaN,"[{""Year"":1990,""percentage_land_encroachment"":0..."
3,AFG,Afghanistan,AFG.4_1,Balkh,4,17253.634668,"POLYGON ((66.42347 35.64057, 66.51625 35.67334...",NaN,"[{""SliceNumber"":26,""per_global"":0.05,""SPS_glob...","[{""SliceNumber"":575,""per_global"":1.61,""SPS_glo...",...,1312167.0,201.0,Sparsley or Non vegetated,Warm Temperate Dry,3,"[{""Year"":1990,""percentage_land_encroachment"":2...","[{""Year"":1990,""percentage_land_encroachment"":0...",NaN,NaN,"[{""Year"":1990,""percentage_land_encroachment"":0..."
4,AFG,Afghanistan,AFG.5_1,Bamyan,5,14173.489095,"POLYGON ((66.65279 34.00322, 66.67175 34.03791...",NaN,"[{""SliceNumber"":26,""per_global"":0.03,""SPS_glob...","[{""SliceNumber"":167,""per_global"":0.22,""SPS_glo...",...,432184.6,142.0,Grassland,Cool Temperate Dry,3,NaN,"[{""Year"":1990,""percentage_land_encroachment"":0...",NaN,NaN,NaN


In [ ]:
dff_ctx.to_file(f'{path_out}/gadm1_precalculated_aoi_summaries.geojson',driver='GeoJSON')

Import this new dataframe in AGOL manually either as a new feature layer and create another service from URL to make it whitelisted